In [1]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 6.5 MB/s eta 0:00:01     |██████████████▎                 | 10.8 MB 4.5 MB/s eta 0:00:03
     |████████████████████████████████| 103 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 3.8 MB/s eta 0:00:011
     |████████████████████████████████| 6.3 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 5.0 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=cfb06ee305a4bc0008d85cf697ce23defa37b00367e2aff40f58def8de8779fe
  Stored in directory: /Users/yoiching/Library/Caches/pip/wheels/bb/1c/9c/412ec03f6d5ac7d41f4b965bde3fc0d1bd201da5ba3e2636de
Successfully built smart-open
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


In [2]:
import gensim

In [3]:
import jieba as ja
import jieba.analyse as jia
import jieba.posseg as pseg
from pprint import pprint
import pandas as pd
import re
import pycantonese as pc
import string
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/ten

In [4]:
ws = WS("/Users/yoiching/Downloads/data")
pos = POS("/Users/yoiching/Downloads/data")
ner = NER("/Users/yoiching/Downloads/data")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [6]:
df = pd.read_csv('/Users/yoiching/Downloads/CoFact.csv')
df.head()

,Link,Topic,Label,Reason,ref
0,https://cofacts.g0v.tw/article/2slt5g65d5syi,"敦睦艦隊確診個案到訪地點:\n 更新日指:109,4, 18+\n 1、左營區:漢神巨蛋(添...",❌ Contains misinformation,假的。謠言內提到的許多地點都是不實訊息，也請特別注意裡面有些字體其實並非台灣用字或是牛排寫成...,Referencehttps://www.facebook.com/995878500506...
1,https://cofacts.g0v.tw/article/1i8lr7ufe7qcp,三艘軍艦744人，4/15離開左營海軍基地，4/18全體召回，第一次採檢及18日今分19日新...,⭕ Contains true information,台灣官方表示，已經透過所有官兵的手機，針對其家人或朋友等接觸者發送簡訊，提醒接觸者須自主健康...,Referencehttps://news.ltn.com.tw/news/life/bre...
2,https://cofacts.g0v.tw/article/1vhszuoepjlpj,https://youtu.be/cukrLiPj2Ck金正恩委員長死去【緊急ニュース速報】...,❌ Contains misinformation,合成剪輯過去 2011 年發生的影片片段所做的謠言影片，並非近期的狀況。,Reference【假影片】金正恩委員長死去？謠言！剪接2011金正日葬禮片段\n http...
3,https://cofacts.g0v.tw/article/32rvzmxik5p3z,金正恩腦死 享年36歲，2020真是恐怖的一年\n \n #金正恩指定勞動黨第一副部長金與正...,❌ Contains misinformation,南韓政府收集到的情報以及中共中央對外聯絡消息人士都表示，金正恩並非病危。韓聯社報導，南韓政府...,Referencehttps://tw.appledaily.com/internation...
4,https://cofacts.g0v.tw/article/2nrmw11qaxzij,記得 今天、明天 2天內要去家樂福買東西順便領取1600元政府補助振興卷。切記！切記！不領白...,❌ Contains misinformation,請注意這並不是政府補助的振興卷，內文的今天跟明天2天也並沒有明確定義，實際上是只有4月19日...,Reference家樂福搶先祭出八億振興劵\n https://news.sina.com....


In [7]:
data = df[["Topic", "Reason", "Label"]].copy()
data.head()

,Topic,Reason,Label
0,"敦睦艦隊確診個案到訪地點:\n 更新日指:109,4, 18+\n 1、左營區:漢神巨蛋(添...",假的。謠言內提到的許多地點都是不實訊息，也請特別注意裡面有些字體其實並非台灣用字或是牛排寫成...,❌ Contains misinformation
1,三艘軍艦744人，4/15離開左營海軍基地，4/18全體召回，第一次採檢及18日今分19日新...,台灣官方表示，已經透過所有官兵的手機，針對其家人或朋友等接觸者發送簡訊，提醒接觸者須自主健康...,⭕ Contains true information
2,https://youtu.be/cukrLiPj2Ck金正恩委員長死去【緊急ニュース速報】...,合成剪輯過去 2011 年發生的影片片段所做的謠言影片，並非近期的狀況。,❌ Contains misinformation
3,金正恩腦死 享年36歲，2020真是恐怖的一年\n \n #金正恩指定勞動黨第一副部長金與正...,南韓政府收集到的情報以及中共中央對外聯絡消息人士都表示，金正恩並非病危。韓聯社報導，南韓政府...,❌ Contains misinformation
4,記得 今天、明天 2天內要去家樂福買東西順便領取1600元政府補助振興卷。切記！切記！不領白...,請注意這並不是政府補助的振興卷，內文的今天跟明天2天也並沒有明確定義，實際上是只有4月19日...,❌ Contains misinformation


In [8]:
file = open("/Users/yoiching/Downloads/stop_word_zh.txt", "r")
stopwords = file.read().split('\n')
file.close()

In [9]:
def remove_bad_string(sentence):
    # remove old style retweet text "RT"
    sentence = re.sub(r'^RT[\s]+', '', sentence)

    # remove hyperlinks
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
    
    #sentence = re.sub(r'#', '', sentence)
    
    sentence = re.sub(r'\s+', '', sentence)
    #remove space
    sentence = re.sub(r' ', '', sentence)
    #1,600 -> 1600
    sentence = re.sub(r',', '', sentence)
    
    return sentence                    

def filter_stopword(sentence):
    
    filter_word=[]
    filter_list=[string.punctuation, stopwords]
    
    for word in sentence:
        for filter_words in filter_list:
            if word in filter_words:
                break
            if word == '\n':
                break
            # 7-11/number of cases
            #if word.isnumeric():
            #    break
        else:
            filter_word.append(word)
    
    return filter_word

In [10]:
def ckip_seg(sentence):
    sentence = remove_bad_string(sentence)
    sentence = ws([sentence])
    sentence = sentence[0]
    sentence = filter_stopword(sentence)
    return sentence

In [49]:
df = pd.read_csv("/Users/yoiching/Downloads/cofact_label.csv")
df.head()

,Unnamed: 0,topic,政治,健康
0,0,"敦睦艦隊確診個案到訪地點:\n 更新日指:109,4, 18+\n 1、左營區:漢神巨蛋(添...",0,1
1,1,三艘軍艦744人，4/15離開左營海軍基地，4/18全體召回，第一次採檢及18日今分19日新...,0,1
2,2,https://youtu.be/cukrLiPj2Ck金正恩委員長死去【緊急ニュース速報】...,1,0
3,3,金正恩腦死 享年36歲，2020真是恐怖的一年\n \n #金正恩指定勞動黨第一副部長金與正...,1,0
4,4,記得 今天、明天 2天內要去家樂福買東西順便領取1600元政府補助振興卷。切記！切記！不領白...,1,0


In [50]:
df = df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)

In [73]:
df["Cat"] = ""
for i in range(len(df)):
    if df["政治"][i] == 1:
        df["Cat"][i] = "政治"
    elif df["健康"][i] == 1:
        df["Cat"][i] = "健康"
    else:
        df["Cat"][i] = "其他"
df.head()

/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,topic,政治,健康,Cat,Tokenized
0,"敦睦艦隊確診個案到訪地點:\n 更新日指:109,4, 18+\n 1、左營區:漢神巨蛋(添...",0,1,健康,敦睦 艦隊 確診 個案 到訪 地點 更新 日指 109418 +1 左營區 漢神 巨蛋 添好...
1,三艘軍艦744人，4/15離開左營海軍基地，4/18全體召回，第一次採檢及18日今分19日新...,0,1,健康,艘 軍艦 744 人 4/15 離開 左營 海軍 基地 4/18 全體 召回 第一 次 採檢...
2,https://youtu.be/cukrLiPj2Ck金正恩委員長死去【緊急ニュース速報】...,1,0,政治,🔽 サ ブ チ ャ ン ネ ル か み な り お や じ の 楽屋 裏 🔽 か み な り...
3,金正恩腦死 享年36歲，2020真是恐怖的一年\n \n #金正恩指定勞動黨第一副部長金與正...,1,0,政治,金正恩 腦死 享年 36 歲 2020 真是 恐怖 年 金正恩 指定 勞動黨 第一 副部長 ...
4,記得 今天、明天 2天內要去家樂福買東西順便領取1600元政府補助振興卷。切記！切記！不領白...,1,0,政治,記得 今天 明天 天 家樂福 買 東西 順便 領取 1600 元 政府 補助 振興卷 切記 ...


In [56]:
df["Tokenized"] = df.topic.apply(ckip_seg)
df.head()

,topic,政治,健康,Cat,Tokenized
0,"敦睦艦隊確診個案到訪地點:\n 更新日指:109,4, 18+\n 1、左營區:漢神巨蛋(添...",0,1,健康,"[敦睦, 艦隊, 確診, 個案, 到訪, 地點, 更新, 日指, 109418, +1, 左..."
1,三艘軍艦744人，4/15離開左營海軍基地，4/18全體召回，第一次採檢及18日今分19日新...,0,1,健康,"[艘, 軍艦, 744, 人, 4/15, 離開, 左營, 海軍, 基地, 4/18, 全體..."
2,https://youtu.be/cukrLiPj2Ck金正恩委員長死去【緊急ニュース速報】...,1,0,政治,"[🔽, サ, ブ, チ, ャ, ン, ネ, ル, か, み, な, り, お, や, じ, ..."
3,金正恩腦死 享年36歲，2020真是恐怖的一年\n \n #金正恩指定勞動黨第一副部長金與正...,1,0,政治,"[金正恩, 腦死, 享年, 36, 歲, 2020, 真是, 恐怖, 年, 金正恩, 指定,..."
4,記得 今天、明天 2天內要去家樂福買東西順便領取1600元政府補助振興卷。切記！切記！不領白...,1,0,政治,"[記得, 今天, 明天, 天, 家樂福, 買, 東西, 順便, 領取, 1600, 元, 政..."


In [58]:
for i in range(len(df)):
    df["Tokenized"][i] = " ".join(df["Tokenized"][i])
df.head()

/Users/yoiching/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,topic,政治,健康,Cat,Tokenized
0,"敦睦艦隊確診個案到訪地點:\n 更新日指:109,4, 18+\n 1、左營區:漢神巨蛋(添...",0,1,健康,敦睦 艦隊 確診 個案 到訪 地點 更新 日指 109418 +1 左營區 漢神 巨蛋 添好...
1,三艘軍艦744人，4/15離開左營海軍基地，4/18全體召回，第一次採檢及18日今分19日新...,0,1,健康,艘 軍艦 744 人 4/15 離開 左營 海軍 基地 4/18 全體 召回 第一 次 採檢...
2,https://youtu.be/cukrLiPj2Ck金正恩委員長死去【緊急ニュース速報】...,1,0,政治,🔽 サ ブ チ ャ ン ネ ル か み な り お や じ の 楽屋 裏 🔽 か み な り...
3,金正恩腦死 享年36歲，2020真是恐怖的一年\n \n #金正恩指定勞動黨第一副部長金與正...,1,0,政治,金正恩 腦死 享年 36 歲 2020 真是 恐怖 年 金正恩 指定 勞動黨 第一 副部長 ...
4,記得 今天、明天 2天內要去家樂福買東西順便領取1600元政府補助振興卷。切記！切記！不領白...,1,0,政治,記得 今天 明天 天 家樂福 買 東西 順便 領取 1600 元 政府 補助 振興卷 切記 ...


**Vectorize the text** \
The first parameter is the max_features parameter, which is set to 1500. This is because when you convert words to numbers using the bag of words approach, all the unique words in all the documents are converted into features. All the documents can contain tens of thousands of unique words. But the words that have a very low frequency of occurrence are unusually not a good parameter for classifying documents. 

The next parameter is min_df and it has been set to 5. This corresponds to the minimum number of documents that should contain this feature. So we only include those words that occur in at least 5 documents. Similarly, for the max_df, feature the value is set to 0.7; in which the fraction corresponds to a percentage. Here 0.7 means that we should include only those words that occur in a maximum of 70% of all the documents

In [129]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.8, stop_words=stopwords)
X = vectorizer.fit_transform(df["Tokenized"]).toarray()
# The fit_transform function of the CountVectorizer class converts text documents into corresponding numeric features.

In [130]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [131]:
y = df["Cat"]

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [133]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [134]:
y_pred = classifier.predict(X_test)

In [135]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[18  0  2]
 [ 2  0  0]
 [ 2  1  5]]
              precision    recall  f1-score   support

          健康       0.82      0.90      0.86        20
          其他       0.00      0.00      0.00         2
          政治       0.71      0.62      0.67         8

    accuracy                           0.77        30
   macro avg       0.51      0.51      0.51        30
weighted avg       0.74      0.77      0.75        30

0.7666666666666667


In [136]:
y_test, y_pred

(26    健康
 84    政治
 2     政治
 55    健康
 68    健康
 81    其他
 16    健康
 78    政治
 54    健康
 66    健康
 53    健康
 89    健康
 71    健康
 13    政治
 7     健康
 30    健康
 22    健康
 24    健康
 33    健康
 8     健康
 43    健康
 62    健康
 3     政治
 86    健康
 45    健康
 48    其他
 6     政治
 96    政治
 80    政治
 75    健康
 Name: Cat, dtype: object,
 array(['政治', '政治', '健康', '健康', '健康', '健康', '健康', '政治', '政治', '健康', '健康',
        '健康', '健康', '其他', '健康', '健康', '健康', '健康', '健康', '健康', '健康', '健康',
        '健康', '健康', '健康', '健康', '政治', '政治', '政治', '健康'], dtype=object))

In [137]:
import pickle

save the model

In [138]:
with open('text_classifier', 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

load the model

In [139]:
with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

In [140]:
y_pred2 = model.predict(X_test)

print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

[[18  0  2]
 [ 2  0  0]
 [ 2  1  5]]
              precision    recall  f1-score   support

          健康       0.82      0.90      0.86        20
          其他       0.00      0.00      0.00         2
          政治       0.71      0.62      0.67         8

    accuracy                           0.77        30
   macro avg       0.51      0.51      0.51        30
weighted avg       0.74      0.77      0.75        30

0.7666666666666667
